In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy.fft import fft2, ifft2

In [5]:
Nx = 30
Ny = 30
Lx = 30
Ly = 30
dx = Lx / Nx
dy = Ly / Ny

c = np.ones((Nx, Ny))
h = np.ones((Nx, Ny))
w = np.ones((Nx, Ny))

Dc = 1
Dh = 1
Dw = 1

A = 1
B = 1

ck = fft2(c)
hk = fft2(h)
wk = fft2(w)

kx = np.fft.fftfreq(Nx, dx) * np.pi * 2
ky = np.fft.fftfreq(Ny, dy) * np.pi * 2

for i in range(30):
    ck = A*wk - B*ck*hk**2 + Dc * (kx**2 + ky**2)*ck + ck
    hk = B*ck*hk**2 - hk*wk + Dc * (kx**2 + ky**2)*hk + hk
    wk = hk*wk - A*wk + Dw * (kx**2 + ky**2)*wk + wk
    plt.plot()

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


/var/folders/s0/cf8vcvv57gz3gzbb4w_n9jm00000gn/T/ipykernel_71766/449042339.py:28: RuntimeWarning: overflow encountered in multiply
  hk = B*ck*hk**2 - hk*wk + Dc * (kx**2 + ky**2)*hk + hk
/var/folders/s0/cf8vcvv57gz3gzbb4w_n9jm00000gn/T/ipykernel_71766/449042339.py:29: RuntimeWarning: invalid value encountered in multiply
  wk = hk*wk - A*wk + Dw * (kx**2 + ky**2)*wk + wk
/var/folders/s0/cf8vcvv57gz3gzbb4w_n9jm00000gn/T/ipykernel_71766/449042339.py:27: RuntimeWarning: invalid value encountered in multiply
  ck = A*wk - B*ck*hk**2 + Dc * (kx**2 + ky**2)*ck + ck
/var/folders/s0/cf8vcvv57gz3gzbb4w_n9jm00000gn/T/ipykernel_71766/449042339.py:27: RuntimeWarning: invalid value encountered in square
  ck = A*wk - B*ck*hk**2 + Dc * (kx**2 + ky**2)*ck + ck
/var/folders/s0/cf8vcvv57gz3gzbb4w_n9jm00000gn/T/ipykernel_71766/449042339.py:28: RuntimeWarning: invalid value encountered in square
  hk = B*ck*hk**2 - hk*wk + Dc * (kx**2 + ky**2)*hk + hk
/var/folders/s0/cf8vcvv57gz3gzbb4w_n9jm00000gn/T/ipy